### Importing necessary libraries


In [1]:

import pandas as pd 
import pyodbc
import boto3
import warnings
import redshift_connector
import psycopg2
from io import StringIO # python3; python2: BytesIO 
import boto3

warnings.filterwarnings('ignore')

### Creating a dictionary to connect to the Local Database

In [2]:

DB = {'servername': 'DESKTOP-1OD4A1P\SQLEXPRESS',
      'database': 'Apartment_KK'}

### Creating a connection to the Database

In [3]:


conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=DESKTOP-1OD4A1P\SQLEXPRESS;"
                      "Database=Apartment_KK;"
                      "Trusted_Connection=yes;")

### Creating a dataframe for each tables in the database.

In [4]:


employee = pd.read_sql_query("Select * FROM employee", conn)
parts = pd.read_sql_query("Select * FROM parts", conn)
supplier = pd.read_sql_query("Select * FROM supplier", conn)
partsmaintenance = pd.read_sql_query("Select * FROM Partsmaintenance", conn)
maintenance = pd.read_sql_query("Select * FROM maintenance", conn)
employeemaintenance = pd.read_sql_query("Select * FROM employeemaintenance", conn)
lease = pd.read_sql_query("Select * FROM lease", conn)
unit = pd.read_sql_query("Select * FROM unit", conn)
parking = pd.read_sql_query("Select * FROM parking", conn)
occupants = pd.read_sql_query("Select * FROM occupants", conn)
resident = pd.read_sql_query("Select * FROM resident", conn)
rentpayment = pd.read_sql_query("Select * FROM rentpayment", conn)

### Creating a fact table for rent payments by joining 4 tables Resident, Unit, Lease and RentPayment

In [5]:


fact_bill = pd.read_sql_query("""SELECT
                                    Resident.ResidentID,
                                    Unit.UnitID,
                                    Lease.LeaseID,
                                    RentPayment.PaymentID,
                                    RentPayment.Date,
                                    RentPayment.Amount,
                                    RentPayment.AmountDue
                                FROM
                                    Resident
                                JOIN
                                    Lease 
                                ON
                                    Lease.ResidentID = Resident.ResidentID
                                JOIN
                                    Unit
                                ON
                                    Lease.UnitID = Unit.UnitID
                                JOIN
                                    RentPayment
                                ON
                                    RentPayment.PaymentID = Unit.UnitID""", conn)

### Creating dimension tables

In [6]:


dim_lease = pd.read_sql_query("""SELECT
                                    LeaseID,
                                    StartDate,
                                    EndDate,
                                    MonthlyRent,
                                    SecurityDeposit
                                FROM 
                                    Lease""",conn)

#Creating dimension table for apartment units

dim_unit = pd.read_sql_query("""SELECT
                                    UnitId,
                                    Area,
                                    Type,
                                    Baths,
                                    Bed,
                                    Laundry,
                                    ParkingNo 
                                FROM 
                                    Unit;""", conn)

#Creating dimension table for residents

dim_resident = pd.read_sql_query("SELECT * FROM Resident",conn)

#Crreating dimension table for payments

dim_payment = pd.read_sql_query("""SELECT
                                        PaymentId,
                                        Type,
                                        Date,
                                        Amount,
                                        AmountDue 
                                    FROM 
                                        RentPayment""",conn)

#Creating a dimension for employees

dim_employee = pd.read_sql_query("SELECT * FROM Employee;", conn)

#Creating a dimension for parts

dim_parts = pd.read_sql_query("""SELECT 
                                    PartId,
                                    Name,
                                    UnitPRice,
                                    QuantityAvailable
                                FROM 
                                    Parts; """,conn)

### Creating a fact table for maintenance joining 3 tables

In [7]:


fact_maintenance = pd.read_sql_query("""SELECT 
                                            Maintenance.UnitID,
                                            EmployeeMaintenance.EmployeeID,
                                            PartsMaintenance.PartID,
                                            Maintenance.Type,
                                            Maintenance.ReportDate,
                                            Maintenance.StartDate,
                                            Maintenance.CompletionDate
                                        FROM 
                                            EmployeeMaintenance
                                        JOIN 
                                            Maintenance
                                        ON 
                                            Maintenance.MaintenanceID = EmployeeMaintenance.EmployeeID
                                        JOIN 
                                            PartsMaintenance
                                        ON 
                                            PartsMaintenance.MaintenanceId = Maintenance.MaintenanceID; """,conn)

### Hiding Aws Access Key and Secret Access Key before connection

In [9]:
import getpass

In [10]:
AWS_ACCESS_KEY = getpass.getpass('Enter AWS ACCESS KEY ')
AWS_SECRET_KEY = getpass.getpass('ENTER AWS SERCRET KEY')
AWS_REGION = "us-east-1"

Enter AWS ACCESS KEY ········
ENTER AWS SERCRET KEY········


### Connection to s3 bucket in AWS.

In [11]:

s3_client = boto3.client("s3",
aws_access_key_id = AWS_ACCESS_KEY,
aws_secret_access_key = AWS_SECRET_KEY,
region_name = AWS_REGION,)

### Uploading all the facts and dimension tables in s3 bucket.

In [12]:

bucket = 'kshitiz-project1' # already created on S3

#fact_bill

csv_buffer = StringIO()
fact_bill.to_csv(csv_buffer)
response = s3_client.put_object(Bucket = bucket, Key = "Apartment/fact_bill.csv", Body = csv_buffer.getvalue())


In [15]:
#dim_lease

csv_buffer = StringIO()
dim_lease.to_csv(csv_buffer)
response = s3_client.put_object(Bucket = bucket, Key = "Apartment/dim_lease.csv", Body = csv_buffer.getvalue())



In [16]:
#dim_unit

dim_unit.to_csv(csv_buffer)
response = s3_client.put_object(Bucket = bucket, Key = "Apartment/dim_unit.csv", Body = csv_buffer.getvalue())


In [17]:
#dim_resident

dim_resident.to_csv(csv_buffer)
response = s3_client.put_object(Bucket = bucket, Key = "Apartment/dim_resident.csv", Body = csv_buffer.getvalue())


In [18]:
#dim_payment

dim_payment.to_csv(csv_buffer)
response = s3_client.put_object(Bucket = bucket, Key = "Apartment/dim_payment.csv", Body = csv_buffer.getvalue())


In [19]:
#fact_maintenance 

fact_maintenance.to_csv(csv_buffer)
response = s3_client.put_object(Bucket = bucket, Key = "Apartment/fact_maintenance.csv", Body = csv_buffer.getvalue())


In [20]:
#dim_employee

dim_employee.to_csv(csv_buffer)
response = s3_client.put_object(Bucket = bucket, Key = "Apartment/dim_employee.csv", Body = csv_buffer.getvalue())


In [21]:
#dim_parts

dim_parts.to_csv(csv_buffer)
response = s3_client.put_object(Bucket = bucket, Key = "Apartment/dim_parts.csv", Body = csv_buffer.getvalue())


### Creating a database schema for all the facts and dimension tables.

In [22]:

fact_bill_sql = pd.io.sql.get_schema(fact_bill.reset_index(),'fact_bill')
print(''.join(fact_bill_sql))

CREATE TABLE "fact_bill" (
"index" INTEGER,
  "ResidentID" INTEGER,
  "UnitID" INTEGER,
  "LeaseID" INTEGER,
  "PaymentID" INTEGER,
  "Date" DATE,
  "Amount" REAL,
  "AmountDue" REAL
)


In [24]:
dim_lease_sql = pd.io.sql.get_schema(dim_lease.reset_index(),'dim_lease')
print(''.join(dim_lease_sql))

CREATE TABLE "dim_lease" (
"index" INTEGER,
  "LeaseID" INTEGER,
  "StartDate" DATE,
  "EndDate" DATE,
  "MonthlyRent" REAL,
  "SecurityDeposit" REAL
)


In [25]:
dim_unit_sql = pd.io.sql.get_schema(dim_unit.reset_index(),'dim_unit')
print(''.join(dim_unit_sql))

CREATE TABLE "dim_unit" (
"index" INTEGER,
  "UnitId" INTEGER,
  "Area" REAL,
  "Type" TEXT,
  "Baths" REAL,
  "Bed" REAL,
  "Laundry" INTEGER,
  "ParkingNo" REAL
)


In [26]:
dim_resident_sql = pd.io.sql.get_schema(dim_resident.reset_index(),'dim_resident')
print(''.join(dim_resident_sql))

CREATE TABLE "dim_resident" (
"index" INTEGER,
  "ResidentID" INTEGER,
  "FirstName" TEXT,
  "LastName" TEXT,
  "EmailAddress" TEXT,
  "Phone" TEXT,
  "PermanentAddress" TEXT
)


In [27]:
dim_payment_sql = pd.io.sql.get_schema(dim_payment.reset_index(),'dim_payment')
print(''.join(dim_payment_sql))

CREATE TABLE "dim_payment" (
"index" INTEGER,
  "PaymentId" INTEGER,
  "Type" TEXT,
  "Date" DATE,
  "Amount" REAL,
  "AmountDue" REAL
)


In [28]:
fact_maintenance_sql = pd.io.sql.get_schema(fact_maintenance.reset_index(),'fact_maintenance')
print(''.join(fact_maintenance_sql))

CREATE TABLE "fact_maintenance" (
"index" INTEGER,
  "UnitID" REAL,
  "EmployeeID" INTEGER,
  "PartID" INTEGER,
  "Type" TEXT,
  "ReportDate" DATE,
  "StartDate" DATE,
  "CompletionDate" DATE
)


In [29]:
dim_employee_sql = pd.io.sql.get_schema(dim_employee.reset_index(),'dim_employee')
print(''.join(dim_employee_sql))

CREATE TABLE "dim_employee" (
"index" INTEGER,
  "EmployeeID" INTEGER,
  "FirstName" TEXT,
  "LastName" TEXT,
  "Position" TEXT,
  "Phone" TEXT
)


In [30]:
dim_parts_sql = pd.io.sql.get_schema(dim_parts.reset_index(),'dim_parts')
print(''.join(dim_parts_sql))

CREATE TABLE "dim_parts" (
"index" INTEGER,
  "PartId" INTEGER,
  "Name" TEXT,
  "UnitPRice" REAL,
  "QuantityAvailable" INTEGER
)


### Hiding Redshift Credentials 

In [32]:

host1 = getpass.getpass('Enter host ')
database1 = getpass.getpass('Enter database name')
user1 = getpass.getpass('Enter your password')
password1 = getpass.getpass('Enter your password')

Enter host ········
Enter database name········
Enter your password········
Enter your password········


In [33]:

conn = redshift_connector.connect(
     host = host1,
     database = database1,
     user = user1,
     password  = password1
  )
conn.autocommit = True
cursor = redshift_connector.Cursor = conn.cursor()


### Script that creates the schema and export data from s3 to Redshift data warehouse which is scheduled in AWS glue job.

In [34]:

#fact_bill
cursor.execute("""CREATE TABLE "fact_bill" (
"index" INTEGER,
  "ResidentID" INTEGER,
  "UnitID" INTEGER,
  "LeaseID" INTEGER,
  "PaymentID" INTEGER,
  "Date" DATE,
  "Amount" REAL,
  "AmountDue" REAL
)""")

cursor.execute("""CREATE TABLE "dim_lease" (
"index" INTEGER,
  "LeaseID" INTEGER,
  "StartDate" DATE,
  "EndDate" DATE,
  "MonthlyRent" REAL,
  "SecurityDeposit" REAL
)""")

cursor.execute("""CREATE TABLE "dim_unit" (
"index" INTEGER,
  "UnitId" INTEGER,
  "Area" REAL,
  "Type" TEXT,
  "Baths" REAL,
  "Bed" REAL,
  "Laundry" TEXT,
  "ParkingNo" REAL
)""")

cursor.execute("""CREATE TABLE "dim_resident" (
"index" INTEGER,
  "ResidentID" INTEGER,
  "FirstName" TEXT,
  "LastName" TEXT,
  "EmailAddress" TEXT,
  "Phone" TEXT,
  "PermanentAddress" TEXT
)""")

cursor.execute("""CREATE TABLE "dim_payment" (
"index" INTEGER,
  "PaymentId" INTEGER,
  "Type" TEXT,
  "Date" DATE,
  "Amount" REAL,
  "AmountDue" REAL
) """)

cursor.execute("""CREATE TABLE "fact_maintenance" (
"index" INTEGER,
  "UnitID" REAL,
  "EmployeeID" INTEGER,
  "PartID" INTEGER,
  "Type" TEXT,
  "ReportDate" DATE,
  "ProblemDescription" TEXT,
  "StartDate" DATE,
  "CompletionDate" DATE
)""")

cursor.execute("""CREATE TABLE "dim_employee" (
"index" INTEGER,
  "EmployeeID" INTEGER,
  "FirstName" TEXT,
  "LastName" TEXT,
  "Position" TEXT,
  "Phone" TEXT
)""")

cursor.execute("""CREATE TABLE "dim_parts" (
"index" INTEGER,
  "PartId" INTEGER,
  "Name" TEXT,
  "Description" TEXT,
  "UnitPRice" REAL,
  "QuantityAvailable" REAL
)""")


#Copy from s3 to Redshift

#fact_bill
cursor.execute("""
copy fact_bill 
from 's3://kshitiz-project/Apartment/fact_bill.csv'
credentials 'aws_iam_role=arn:aws:iam::829978880207:role/s3-redshift'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

#dim_lease
cursor.execute("""
copy dim_lease 
from 's3://kshitiz-project/Apartment/dim_lease.csv'
credentials 'aws_iam_role=arn:aws:iam::829978880207:role/s3-redshift'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

#dim_resident
cursor.execute("""
copy dim_resident 
from 's3://kshitiz-project/Apartment/dim_resident.csv'
credentials 'aws_iam_role=arn:aws:iam::829978880207:role/s3-redshift'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

#dim_payment
cursor.execute("""
copy dim_payment 
from 's3://kshitiz-project/Apartment/dim_payment.csv'
credentials 'aws_iam_role=arn:aws:iam::829978880207:role/s3-redshift'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

#dim_unit
cursor.execute("""
copy dim_unit 
from 's3://kshitiz-project/Apartment/dim_unit.csv'
credentials 'aws_iam_role=arn:aws:iam::829978880207:role/s3-redshift'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

#dim_employee
cursor.execute("""
copy dim_employee 
from 's3://kshitiz-project/Apartment/dim_employee.csv'
credentials 'aws_iam_role=arn:aws:iam::829978880207:role/s3-redshift'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

#dim_parts
cursor.execute("""
copy dim_parts 
from 's3://kshitiz-project/Apartment/dim_parts.csv'
credentials 'aws_iam_role=arn:aws:iam::829978880207:role/s3-redshift'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")

#fact_maintenance
cursor.execute("""
copy dim_resident 
from 's3://kshitiz-project/Apartment/dim_resident.csv'
credentials 'aws_iam_role=arn:aws:iam::829978880207:role/s3-redshift'
delimiter ','
region 'us-east-1'
IGNOREHEADER 1
""")
